In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler as MMS
import joblib
from tensorflow.keras.models import model_from_json

In [98]:
def Scale(x):
    scaler = MMS() 
    data = scaler.fit_transform(x)
    return data, scaler
def InverseScale(data, scaler):
    predictions = scaler.inverse_transform(data.reshape(-1, 1)).flatten()
    return predictions

In [110]:
def Create_data(dataX, m):
    if(m == 'X'):
        negative_press = dataX.drop('Date_time', axis =1).min()[dataX.drop('Date_time', axis =1).min() < 0].index.tolist()
        dataX[negative_press] = dataX[negative_press].apply(lambda x:x+5)
#     ------------------------------------------
        dataX.set_index("Date_time", inplace=True) #set index
#     ------------------------------------------
        features = dataX.columns
        control = dataX["BOO_TM2_Pressure"]
        dataX_scaler, scaler = Scale(dataX[features])
        control, scaler = Scale(control.values.reshape(-1, 1))
        dataX = pd.DataFrame(columns=features, data=dataX_scaler, index = dataX.index)
        X = []
        X.append(dataX.values)
        X = np.array(X)
#         window = X.shape[1]
#         feat = X.shape[2]
#         X = X.reshape(window, feat)
        return X, scaler

In [42]:
from tensorflow.keras.models import load_model
loaded_model = load_model('model2_0.h5')

In [111]:
# thử
ex = pd.read_csv('ALL.csv')
dataX_scaler, scaler = Create_data(ex[:288], 'X')
y = loaded_model.predict(dataX_scaler).flatten()
InverseScale(y, scaler)

1/1 [==============================] - 0s 33ms/step


array([12.740962], dtype=float32)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
import pywt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import tensorflow as tf
from keras.optimizers import SGD, Adam
from tensorflow import keras
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredError

In [2]:
#doc dataset
df = pd.read_csv("ALL.csv")
df.head()
print(df.isnull().sum())
#xử lí giá trị âm
colum_press = ['BO1016_Pressure(m)', 'BO1017_Pressure(m)', 'BO1021_Pressure', 'BO1022_Pressure(m)', 'BO1029_Pressure(m)', 'NB1019_Pressure', 'TD1029_Pressure', 'TD1043_Press(m)']
df[colum_press] = df[colum_press].apply(lambda x:x+5)
df.describe()
#tách tập trạng thái và tập điều khiển
df.set_index("Date_time", inplace=True) #set index
control = df["BOO_TM2_Pressure"]
state = df
# state = df.drop([ "BOO_TM2_Pressure"], axis = 1)
features = df.columns
scaler =MMS() #scale data
state_scaler = scaler.fit_transform(state[features])
state = pd.DataFrame(columns=features, data=state_scaler, index = df["BOO_TM2_Pressure"].index)
state = state.drop([ "BOO_TM2_Pressure"], axis = 1)
control_scaler = scaler.fit_transform(control.values.reshape(-1, 1)) 
control = pd.DataFrame(columns=["BOO_TM2_Pressure"], data=control_scaler, index = df.index)
# check
print(state.head())
print(control.head())
# tạo biến thử
dense_act = ['ReLU', 'elu', 'tanh', 'sigmoid', 'selu'] 
recur_act = ['ReLU', 'elu', 'tanh', 'sigmoid', 'selu']
epoch = 100
# epoch = [100, 256, 500]
learning_r = 0.001
# learning_r = [0.0005, 0.001, 0.002] 
ls = 'mae'
# ls = ['mae', 'mse']
'''opt1 = SGD()
opt2 = Adam()'''
print('yes')
kind = []
sc = []

Date_time             0
BO1013_FLOW1          0
BO1016_Pressure(m)    0
BO1017_Pressure(m)    0
BO1021_FLOW1)         0
BO1021_Pressure       0
BO1022_flow1          0
BO1022_Pressure(m)    0
BO1017_FLOW1          0
BO1029_FLOW1          0
BO1029_Pressure(m)    0
NB1013_FLOW1          0
NB1017_FLOW1          0
NB1017_Pressure       0
NB1019_FLOW1          0
NB1019_Pressure       0
NB1020_FLOW1          0
NB1021_FLOW1          0
NB1022_FLOW1          0
TD1029_flow1          0
TD1029_Pressure       0
TD1043_FLOW1          0
TD1043_Press(m)       0
NMNBOOTM1_FLOW1       0
BOO_TM2_Pressure      0
dtype: int64
                BO1013_FLOW1  BO1016_Pressure(m)  BO1017_Pressure(m)  \
Date_time                                                              
9/15/2023 0:00      0.463406            0.616835            0.374450   
9/15/2023 0:15      0.450752            0.622623            0.389156   
9/15/2023 0:30      0.439808            0.627547            0.402763   
9/15/2023 0:45      0.43023

In [3]:
#  Tạo các tập X và Y
X = []
Y = []
# Loop qua các hàng của dataframe để lấy các cửa sổ thời gian t-5 đến t và giá trị tại t+1 cho Y
for i in range(288, len(df) - 1):
    X.append(state.iloc[i-288:i].values)  # 
    Y.append(control.iloc[i])  # Lấy giá trị t của cột cuối cùng

X = np.array(X)
Y = np.array(Y)
X[0],Y[0]
X.shape, Y.shape

((10175, 288, 23), (10175, 3, 1))

In [4]:
X_state_tr, X_state_te, y_state_tr, y_state_te = train_test_split(X, Y, test_size = 0.25, random_state = 25)
X_state_tr.shape , y_state_tr.shape

((7631, 288, 23), (7631, 3, 1))

In [5]:
opt1 = Adam(learning_rate= 0.001)
model1 = Sequential([
        tf.keras.layers.LSTM(64, recurrent_activation = 'tanh', input_shape = [288, 23]),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation = 'elu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation = 'elu'),
#         tf.keras.layers.Dense(8, activation = 'elu'),
        tf.keras.layers.Dense(3)
                         ])
model1.compile(optimizer = opt1, loss = ls, )
apply1 = model1.fit(X_state_tr, y_state_tr, batch_size = 1024, epochs = 500, validation_split = 0.2)
score1 = model1.evaluate(X_state_te, y_state_te)

Epoch 1/500
6/6 [==============================] - 28s 3s/step - loss: 0.2394 - val_loss: 0.1336
Epoch 2/500
6/6 [==============================] - 18s 3s/step - loss: 0.1483 - val_loss: 0.1313
Epoch 3/500
6/6 [==============================] - 16s 3s/step - loss: 0.1319 - val_loss: 0.1236
Epoch 4/500
6/6 [==============================] - 21s 3s/step - loss: 0.1263 - val_loss: 0.1129
Epoch 5/500
6/6 [==============================] - 19s 3s/step - loss: 0.1166 - val_loss: 0.1043
Epoch 6/500
6/6 [==============================] - 17s 3s/step - loss: 0.1057 - val_loss: 0.0881
Epoch 7/500
6/6 [==============================] - 15s 3s/step - loss: 0.0924 - val_loss: 0.0763
Epoch 8/500
6/6 [==============================] - 17s 3s/step - loss: 0.0807 - val_loss: 0.0645
Epoch 9/500
6/6 [==============================] - 17s 3s/step - loss: 0.0712 - val_loss: 0.0605
Epoch 10/500
6/6 [==============================] - 16s 3s/step - loss: 0.0683 - val_loss: 0.0581
Epoch 11/500
6/6 [===========

6/6 [==============================] - 13s 2s/step - loss: 0.0322 - val_loss: 0.0335
Epoch 168/500
6/6 [==============================] - 14s 2s/step - loss: 0.0328 - val_loss: 0.0324
Epoch 169/500
6/6 [==============================] - 14s 2s/step - loss: 0.0327 - val_loss: 0.0331
Epoch 170/500
6/6 [==============================] - 13s 2s/step - loss: 0.0332 - val_loss: 0.0315
Epoch 171/500
6/6 [==============================] - 13s 2s/step - loss: 0.0324 - val_loss: 0.0323
Epoch 172/500
6/6 [==============================] - 14s 2s/step - loss: 0.0330 - val_loss: 0.0323
Epoch 173/500
6/6 [==============================] - 12s 2s/step - loss: 0.0326 - val_loss: 0.0310
Epoch 174/500
6/6 [==============================] - 12s 2s/step - loss: 0.0320 - val_loss: 0.0314
Epoch 175/500
6/6 [==============================] - 12s 2s/step - loss: 0.0320 - val_loss: 0.0324
Epoch 176/500
6/6 [==============================] - 12s 2s/step - loss: 0.0320 - val_loss: 0.0302
Epoch 177/500
6/6 [=====

Epoch 250/500
6/6 [==============================] - 19s 3s/step - loss: 0.0258 - val_loss: 0.0248
Epoch 251/500
6/6 [==============================] - 18s 3s/step - loss: 0.0260 - val_loss: 0.0241
Epoch 252/500
6/6 [==============================] - 19s 3s/step - loss: 0.0258 - val_loss: 0.0247
Epoch 253/500
6/6 [==============================] - 18s 3s/step - loss: 0.0258 - val_loss: 0.0245
Epoch 254/500
6/6 [==============================] - 18s 3s/step - loss: 0.0264 - val_loss: 0.0247
Epoch 255/500
6/6 [==============================] - 17s 3s/step - loss: 0.0261 - val_loss: 0.0265
Epoch 256/500
6/6 [==============================] - 19s 3s/step - loss: 0.0267 - val_loss: 0.0263
Epoch 257/500
6/6 [==============================] - 19s 3s/step - loss: 0.0264 - val_loss: 0.0256
Epoch 258/500
6/6 [==============================] - 19s 3s/step - loss: 0.0268 - val_loss: 0.0259
Epoch 259/500
6/6 [==============================] - 21s 3s/step - loss: 0.0271 - val_loss: 0.0251
Epoch 260/

Epoch 415/500
6/6 [==============================] - 13s 2s/step - loss: 0.0222 - val_loss: 0.0215
Epoch 416/500
6/6 [==============================] - 13s 2s/step - loss: 0.0234 - val_loss: 0.0226
Epoch 417/500
6/6 [==============================] - 13s 2s/step - loss: 0.0238 - val_loss: 0.0220
Epoch 418/500
6/6 [==============================] - 13s 2s/step - loss: 0.0227 - val_loss: 0.0220
Epoch 419/500
6/6 [==============================] - 13s 2s/step - loss: 0.0219 - val_loss: 0.0212
Epoch 420/500
6/6 [==============================] - 13s 2s/step - loss: 0.0216 - val_loss: 0.0206
Epoch 421/500
6/6 [==============================] - 13s 2s/step - loss: 0.0211 - val_loss: 0.0216
Epoch 422/500
6/6 [==============================] - 13s 2s/step - loss: 0.0209 - val_loss: 0.0215
Epoch 423/500
6/6 [==============================] - 14s 2s/step - loss: 0.0210 - val_loss: 0.0207
Epoch 424/500
6/6 [==============================] - 13s 2s/step - loss: 0.0209 - val_loss: 0.0201
Epoch 425/